# Preprocesado datos COVID-19

In [2]:
import numpy as np
import pandas as pd
import wget
import os

In [3]:
 # datos del Centro Nacional de Epidemiología
urlfile = "https://cnecovid.isciii.es/covid19/resources/casos_hosp_uci_def_sexo_edad_provres.csv"
name = os.getcwd() + "\\datos\\datos_covid_originales.csv"

if os.path.exists(name):
        os.remove(name)
        
wget.download(urlfile, name)

URLError: <urlopen error [Errno 11001] getaddrinfo failed>

### datos_covid_tratados

In [ ]:
datos_covid = pd.read_csv("datos/datos_covid_originales.csv")
datos_covid.tail(5)

El archivo provincias_iso relaciona el nombre de cada provincia con su codigo iso

In [4]:
codigo_iso = pd.read_csv("datos/provincias_iso.csv", sep=";")
codigo_iso['codigo_iso']=codigo_iso['codigo_iso'].apply(lambda x: x.split("-")[1])

codigo_prov = pd.read_csv("datos/cod_prov.csv", sep=";")
codigo_prov = codigo_prov.merge(codigo_iso, on="nombre_provincia", how="left")
codigo_prov = codigo_prov.rename(columns={'codigo_iso':'provincia_iso','CODIGO':'cpro'})

codigo_ccaa = pd.read_csv("datos/cod_ccaa.csv", sep=";")
codigo_ccaa = codigo_ccaa.rename(columns={'CPRO':'cpro'})
codigo_ccaa = codigo_ccaa.merge(codigo_prov, on= 'cpro')
codigo_ccaa = codigo_ccaa.rename(columns={'CODAUTO':'codauto',
                                          'Comunidad Autónoma':'ccaa',
                                          'Provincia':'provincia'})[['codauto','cpro','ccaa','provincia','provincia_iso']]
codigo_ccaa.head(5)

,codauto,cpro,ccaa,provincia,provincia_iso
0,1,4,Andalucía,Almería,AL
1,1,11,Andalucía,Cádiz,CA
2,1,14,Andalucía,Córdoba,CO
3,1,18,Andalucía,Granada,GR
4,1,21,Andalucía,Huelva,H


Obtenemos la población total de cada provincia,  nos quedamos con el codigo y la población total

In [5]:
poblacion_total = pd.read_csv("datos/poblacion_total_provincias.csv", sep=";")
poblacion_total['Provincias'] = poblacion_total['Provincias'].apply(lambda x:x[:2])
poblacion_total = poblacion_total.drop(columns=['Sexo'])
poblacion_total = poblacion_total.rename(columns={'Provincias':'cpro', 'Total':'poblacion_total'})
poblacion_total.head(5)

,cpro,poblacion_total
0,02,388.270
1,03,1.879.888
2,04,727.945
3,01,333.940
4,33,1.018.784


Unimos los datos de las CCAA con los datos del covid.

In [6]:
datos = datos_covid.merge(codigo_ccaa, on='provincia_iso')
datos['sexo']=datos['sexo'].apply(lambda x: "Hombres" if x=="H" else 
                                            "Mujeres" if x=="M" else "NC")
datos = datos[['fecha','codauto','cpro','ccaa','provincia','sexo','grupo_edad','num_casos','num_hosp','num_uci','num_def']]


datos.head(3)

,fecha,codauto,cpro,ccaa,provincia,sexo,grupo_edad,num_casos,num_hosp,num_uci,num_def
0,2020-01-01,10,3,Comunitat Valenciana,Alicante/Alacant,Hombres,0-9,0,0,0,0
1,2020-01-01,10,3,Comunitat Valenciana,Alicante/Alacant,Hombres,10-19,0,0,0,0
2,2020-01-01,10,3,Comunitat Valenciana,Alicante/Alacant,Hombres,20-29,0,0,0,0


In [7]:
datos['fecha'] = pd.to_datetime(datos['fecha'])
datos['semana'] = datos['fecha'].apply(lambda x: x.week if x< pd.to_datetime('2021-01-04') else x.week+53)

In [8]:
datos.to_csv('datos/datos_covid_tratados.csv', index=False)

### poblacion_edad_sexo

In [9]:
poblacion_edad_sexo = pd.read_csv("datos/poblacion_sexo_edad_provincias.csv", sep=";")

poblacion_edad_sexo.rename(columns={'provincias':'cpro'}, inplace=True)
poblacion_edad_sexo['cpro']=poblacion_edad_sexo['cpro'].apply(lambda x: x[0:2])
poblacion_edad_sexo['grupo_edad']=poblacion_edad_sexo['grupo_edad'].str.strip().apply(lambda x: 
                                     "0-9" if (x=="De 0 a 4 años" or x=="De 5 a 9 años") else 
                                     "10-19" if (x=="De 10 a 14 años" or x=="De 15 a 19 años") else
                                     "20-29" if (x=="De 20 a 24 años" or x=="De 25 a 29 años") else
                                     "30-39" if (x=="De 30 a 34 años" or x=="De 35 a 39 años") else                              
                                     "40-49" if (x=="De 40 a 44 años" or x=="De 45 a 49 años") else                              
                                     "50-59" if (x=="De 50 a 54 años" or x=="De 55 a 59 años") else
                                     "60-69" if (x=="De 60 a 64 años" or x=="De 65 a 69 años") else
                                     "70-79" if (x=="De 70 a 74 años" or x=="De 75 a 79 años") else "80+")

poblacion_edad_sexo = poblacion_edad_sexo.groupby(['cpro','sexo','grupo_edad'])['poblacion_total'].agg('sum').reset_index()
poblacion_edad_sexo = poblacion_edad_sexo[['sexo','cpro','grupo_edad','poblacion_total']]

poblacion_edad_sexo.head(5)

,sexo,cpro,grupo_edad,poblacion_total
0,Hombres,01,0-9,14.076
1,Hombres,01,10-19,15.101
2,Hombres,01,20-29,12.443
3,Hombres,01,30-39,16.585
4,Hombres,01,40-49,25.571


In [10]:
datos_total = poblacion_edad_sexo.groupby(['cpro','sexo']).agg('sum').rename(columns={'poblacion_total':'poblacion_agregada'})
datos_total.head(4)

poblacion_agregada
cpro sexo                       
01   Hombres             149.174
     Mujeres             153.673
02   Hombres             179.690
     Mujeres             181.269

In [11]:
poblacion_edad_sexo = poblacion_edad_sexo.merge(datos_total, on=['cpro','sexo'], how='left')
poblacion_edad_sexo = poblacion_edad_sexo.assign(porcentaje = lambda x: 
                           poblacion_edad_sexo['poblacion_total']/poblacion_edad_sexo['poblacion_agregada']*100) 

In [12]:
poblacion_edad_sexo['cpro']=poblacion_edad_sexo['cpro'].astype(int) # necesario para futuros merge

poblacion_edad_sexo.head()

,sexo,cpro,grupo_edad,poblacion_total,poblacion_agregada,porcentaje
0,Hombres,1,0-9,14.076,149.174,9.435961
1,Hombres,1,10-19,15.101,149.174,10.123078
2,Hombres,1,20-29,12.443,149.174,8.341266
3,Hombres,1,30-39,16.585,149.174,11.117889
4,Hombres,1,40-49,25.571,149.174,17.141727


In [13]:
poblacion_edad_sexo.to_csv('datos/poblacion_edad_sexo.csv', index=False)

### datos_poblacion_sexos

In [14]:
datos_poblacion_sexos = datos.groupby(['cpro','sexo','grupo_edad'],as_index=False).agg({
    'codauto': np.min, 'provincia': np.min, 'num_casos': np.sum, 'num_def': np.sum})

datos_poblacion_sexos = datos_poblacion_sexos.merge(poblacion_edad_sexo, 
                                                    on=['sexo','cpro','grupo_edad'], 
                                                    how="left").merge(codigo_ccaa[['cpro','ccaa']],
                                                                      on='cpro')
datos_poblacion_sexos.head(4)

,cpro,sexo,grupo_edad,codauto,provincia,num_casos,num_def,poblacion_total,poblacion_agregada,porcentaje,ccaa
0,1,Hombres,0-9,16,Araba/Álava,3325,0,14.076,149.174,9.435961,País Vasco
1,1,Hombres,10-19,16,Araba/Álava,4179,0,15.101,149.174,10.123078,País Vasco
2,1,Hombres,20-29,16,Araba/Álava,4179,0,12.443,149.174,8.341266,País Vasco
3,1,Hombres,30-39,16,Araba/Álava,4172,1,16.585,149.174,11.117889,País Vasco


In [15]:
poblacion_edad_sexo.to_csv('datos/datos_periodo.csv', index=False)